In [4]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/visML_project')

Mounted at /content/drive


In [5]:
import torch
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

True
True


In [6]:
cd BoT-SORT/

/content/drive/MyDrive/visML_project/BoT-SORT


In [ ]:
!pip3 install -r requirements.txt
!pip install onnxruntime==1.12.0
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox
!pip3 install faiss-cpu
!pip3 install faiss-gpu
!python3 setup.py develop
!pip install loguru
!pip install thop
!pip install lap
!pip install yacs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 83.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement onnxruntime==1.8.0 (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.17.0, 1.17.1, 1.17.3)
ERROR: No matching distribution found for on

In [ ]:
cd BoT-SORT/

/content/drive/MyDrive/visML_project/BoT-SORT


In [ ]:
ls '/content/drive/My Drive/visML_project/'

BoT-SORT/                 output_frames/  streetaware_sample_chase_1_sensor_1_left/
bot_sort.ipynb            outputs/        streetaware_sample_chase_1_sensor_1_right/
data_preprocessing.ipynb  output_videos/  streetaware_sample_chase_1_sensor_2_left/
MOT17/                    StreetAware/    streetaware_sample_chase_1_sensor_2_right/


In [ ]:
!python3 tools/track.py "/content/drive/My Drive/visML_project/streetaware_sample_chase_1_sensor_1_right" --default-parameters --fast-reid-weights pretrained/mot17_sbs_S50.pth --benchmark "MOT17" --eval "test" --fp16 --fuse


2024-05-01 14:58:17.410305: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 14:58:17.410384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 14:58:17.552343: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-01 14:58:19.691615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-01 14:58:59.756 | INFO     | __main__:main:228 - Args: Namespace(path='/content/drive/My Drive/visML_project/streetaware_sample_chase_1_sensor_1_right/test/MOT17-01-FRCNN/img1', benchmark='MOT17', split_to_eval='test', exp_file='./yo

In [ ]:
ls 'outputs/botsort/chase_1/sensor_1/right_people_track.txt'


left_people_track.txt  right_people_track.json  right_people_track.txt


In [ ]:
 write_results_score(save_seq_txt, seq_results)

In [ ]:
save_seq_txt


'MOT17-01-FRCNN.txt'

In [ ]:
ls  '/content/drive/My Drive/visML_project/outputs/botsort/chase_1/sensor_1/'

left_people_track.txt  right_people_track.json  right_people_track.txt


In [ ]:
import cv2
import numpy as np
import os

# Load tracking results
track_file = '/content/drive/My Drive/visML_project/outputs/botsort/chase_1/sensor_1/right_people_track.txt'
tracks = {}
with open(track_file, 'r') as file:
    for line in file:
        frame_id, track_id, x, y, w, h, conf, _, _, _ = map(float, line.strip().split(','))
        if frame_id not in tracks:
            tracks[frame_id] = []
        tracks[frame_id].append((track_id, x, y, w, h, conf))

# Assign a unique color to each track_id
color_map = {}
for frame in tracks.values():
    for track_id, *_ in frame:
        if track_id not in color_map:
            color_map[track_id] = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))

# Directory containing frames
frames_dir = '/content/drive/My Drive/visML_project/streetaware_sample_chase_1_sensor_1_right/test/MOT17-01-FRCNN/img1/'

# Output directory
output_dir = '/content/drive/My Drive/visML_project/output_frames/sensor_2_left'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Scale factor for resizing
scale_factor = 0.5

# Process each frame
for frame_file in sorted(os.listdir(frames_dir)):
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)
    if frame is None:
        continue

    # Resize frame
    frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

    frame_id = int(frame_file.split('.')[0])  # assuming frame filenames are like '000001.jpg'
    if frame_id in tracks:
        for track_id, x, y, w, h, conf in tracks[frame_id]:
            # Scale bounding box coordinates and size
            x *= scale_factor
            y *= scale_factor
            w *= scale_factor
            h *= scale_factor
            color = color_map[track_id]
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f'ID: {int(track_id)}, Conf: {conf:.2f}', (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save the frame
    output_path = os.path.join(output_dir, frame_file)
    cv2.imwrite(output_path, frame)


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
import cv2
import numpy as np
import os

# Load tracking results
track_file = '/content/drive/My Drive/visML_project/outputs/right_people_track.txt'
tracks = {}
with open(track_file, 'r') as file:
    for line in file:
        frame_id, track_id, x, y, w, h, conf, _, _, _ = map(float, line.strip().split(','))
        if frame_id not in tracks:
            tracks[frame_id] = []
        tracks[frame_id].append((track_id, x, y, w, h, conf))

# Assign a unique color to each track_id
color_map = {}
for frame in tracks.values():
    for track_id, *_ in frame:
        if track_id not in color_map:
            color_map[track_id] = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))

# Directory containing frames
frames_dir = '/content/drive/My Drive/visML_project/streetaware_sample_chase_1_sensor_2_right/test/MOT17-01-FRCNN/img1/'

# Output directory
output_dir = '/content/drive/My Drive/visML_project/output_frames/sensor_2_right'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Scale factor for resizing
scale_factor = 1

# Process each frame
for frame_file in sorted(os.listdir(frames_dir)):
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)
    if frame is None:
        continue

    # Resize frame
    frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

    frame_id = int(frame_file.split('.')[0])  # assuming frame filenames are like '000001.jpg'
    if frame_id in tracks:
        for track_id, x, y, w, h, conf in tracks[frame_id]:
            # Scale bounding box coordinates and size
            x *= scale_factor
            y *= scale_factor
            w *= scale_factor
            h *= scale_factor
            color = color_map[track_id]
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f'ID: {int(track_id)}, Conf: {conf:.2f}', (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save the frame
    output_path = os.path.join(output_dir, frame_file)
    cv2.imwrite(output_path, frame)


In [ ]:
ls '/content/drive/My Drive/visML_project/StreetAware/sensor_1/'

array.json  left_half.mp4  left_people_cleaned.json     left_quarter.mp4  right.mp4
img_left/   left.json      left_people_cleaned_mot.txt  right_half.mp4    right_people.json
img_right/  left.mp4       left_people.json             right.json        right_quarter.mp4


In [ ]:
json_file_path = 'StreetAware/sensor_1/left_people.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

In [ ]:
ls '/content/drive/My Drive/visML_project/streetaware_sample_chase_1_sensor_2_right/'

test/


In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

# Path to the directory containing frames
frames_dir = '/content/drive/My Drive/visML_project/streetaware_sample_chase_2_sensor_1_right/test/MOT17-01-FRCNN/img1/'

# Path to the detections file
detections_file ='/content/drive/My Drive/visML_project/streetaware_sample_chase_2_sensor_1_right/test/MOT17-01-FRCNN/det1/det.txt'

# Read detections from file
detections = {}
with open(detections_file, 'r') as f:
    for line in f:
        frame_id, _, x, y, w, h, confidence = line.strip().split(',')
        frame_id, x, y, w, h, confidence = int(frame_id), float(x), float(y), float(w), float(h), float(confidence)
        if frame_id not in detections:
            detections[frame_id] = []
        # Scale the bounding box coordinates and dimensions
        scale_factor = 0.5  # Scaling down to 50%
        scaled_x = int(x * scale_factor)
        scaled_y = int(y * scale_factor)
        scaled_w = int(w * scale_factor)
        scaled_h = int(h * scale_factor)
        detections[frame_id].append((scaled_x, scaled_y, scaled_w, scaled_h, confidence))

# Initialize frame counter
frame_counter = 0
max_frames = 10  # Maximum number of frames to process

# Process each frame and draw bounding boxes
for frame_id in sorted(detections.keys()):
    if frame_counter >= max_frames:
        break  # Stop processing after 5 frames

    frame_file = f'{frame_id:06d}.jpg'
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)

    if frame is None:
        print(f"Frame {frame_file} not found.")
        continue

    # Resize the frame
    frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # Draw bounding boxes on the frame
    for x, y, w, h, confidence in detections[frame_id]:
        top_left = (x, y)
        bottom_right = (x + w, y + h)
        color = (0, 255, 0) if confidence >= 0 else (0, 0, 255)  # Green if confidence >= 0, red otherwise
        cv2.rectangle(frame, top_left, bottom_right, color, 2)
        cv2.putText(frame, f'Conf: {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # display the frame
    cv2_imshow(frame)  # Display the frame in Colab
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_counter += 1  # Increment the frame counter

cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
cd

 BoT-SORT/                  MOT17/                StreetAware/
 bot_sort.ipynb            'output_frames (1)'/   streetaware_sample_chase_1_sensor_1_left/
 data_preprocessing.ipynb   outputs/              streetaware_sample_chase_1_sensor_1_right/


In [ ]:
import cv2
import os
import json
from google.colab.patches import cv2_imshow

# Load the JSON data
#with open('/content/drive/My Drive/visML_project/StreetAware/sensor_1/right_people.json', 'r') as file:
with open('/content/drive/My Drive/visML_project/StreetAware/sensor_1/left_people.json', 'r') as file:
    data = json.load(file)

# Directory containing frames
#frames_dir ='/content/drive/My Drive/visML_project/StreetAware/sensor_1/img_right/'
frames_dir ='/content/drive/My Drive/visML_project/StreetAware/sensor_1/img_right/'


# Initialize counter for non-null frames
non_null_counter = 0
max_non_null_frames = 10

# Scaling factor
scale_factor = 0.5  # Reduce size to 50%

# Process each item in the JSON data
for frame_index, frame_data in enumerate(data):
    if frame_data is None:
        continue  # Skip null frames

    # Increment the counter and break if maximum frames are processed
    non_null_counter += 1
    if non_null_counter > max_non_null_frames:
        break

    frame_file = f'{frame_index + 1:06d}.jpg'  # Frame file names are 1-based index
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)

    if frame is None:
        print(f"Frame {frame_file} not found.")
        continue

    # Resize the frame
    frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # Draw bounding boxes
    for box in frame_data['boxes']:
        top_left = (int(box[0][0] * scale_factor), int(box[0][1] * scale_factor))
        bottom_right = (int(box[1][0] * scale_factor), int(box[1][1] * scale_factor))
        cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)  # Green color

    # display the frame
    cv2_imshow(frame)  # Display the frame in Colab
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/visML_project/StreetAware/sensor_1/left_people.json'

In [ ]:
import cv2
import os
import json
from google.colab.patches import cv2_imshow

# Load the JSON data
with open('/content/drive/My Drive/visML_project/StreetAware/sensor_1/left_people.json', 'r') as file:
    data = json.load(file)

# Directory containing frames
frames_dir = '/content/drive/My Drive/visML_project/StreetAware/sensor_1/img_left/'

# Output directory
output_dir = '/content/drive/My Drive/visML_project/output_frames'
os.makedirs(output_dir, exist_ok=True)

# Initialize counter for non-null frames
non_null_counter = 0
max_non_null_frames = 10

# Scaling factor
scale_factor = 0.5  # Reduce size to 50%

# Process each item in the JSON data
for frame_index, frame_data in enumerate(data):
    if frame_data is None:
        continue  # Skip null frames

    # Increment the counter and break if maximum frames are processed
    non_null_counter += 1
    if non_null_counter > max_non_null_frames:
        break

    frame_file = f'{frame_index + 1:06d}.jpg'  # Frame file names are 1-based index
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)

    if frame is None:
        print(f"Frame {frame_file} not found.")
        continue

    # Resize the frame
    frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # Draw bounding boxes and display confidence
    scores = frame_data.get('scores', [])  # Retrieve scores if they exist
    for box_index, box in enumerate(frame_data['boxes']):
        top_left = (int(box[0][0] * scale_factor), int(box[0][1] * scale_factor))
        bottom_right = (int(box[1][0] * scale_factor), int(box[1][1] * scale_factor))

        # Determine the color based on the availability of a score
        if box_index < len(scores):
            color = (0, 255, 0)  # Green for boxes with scores
            score = scores[box_index]
            text = f'{score:.2f}'
        else:
            color = (0, 0, 255)  # Red for boxes without scores
            text = 'N/A'

        cv2.rectangle(frame, top_left, bottom_right, color, 2)
        cv2.putText(frame, text, (top_left[0], top_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save or display the frame
    output_path = os.path.join(output_dir, frame_file)
    cv2.imwrite(output_path, frame)
    cv2_imshow(frame)  # Display the frame in Colab
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
ls output_frames/

sensor_1_left/  sensor_1_right/  sensor_2_left/  sensor_2_right/


In [ ]:
import cv2
import os

# Directory containing frames
frames_dir = 'output_frames/sensor_2_left/'

# Output directory
output_dir = 'output_videos/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Video setup
output_video_path = os.path.join(output_dir, 'sensor_2_left.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 files
fps = 30

# Assume first frame to get video dimensions
first_frame_path = os.path.join(frames_dir, '000001.jpg')
first_frame = cv2.imread(first_frame_path)
frame_height, frame_width = first_frame.shape[:2]
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process each frame
sorted_frame_files = sorted(os.listdir(frames_dir))  # Sort to ensure correct frame order
for frame_file in sorted_frame_files:
    if frame_file.endswith('.jpg'):  # Check for jpeg images
        frame_path = os.path.join(frames_dir, frame_file)
        frame = cv2.imread(frame_path)
        if frame is None:
            continue
        video_writer.write(frame)

# Release the VideoWriter
video_writer.release()
print(f"Video saved to {output_video_path}")


Video saved to output_videos/sensor_2_left.mp4


In [ ]:
import cv2
import os

# Directory containing frames
frames_dir = 'output_frames/sensor_2_right/'

# Output directory
output_dir = 'output_videos/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Video setup
output_video_path = os.path.join(output_dir, 'sensor_2_right.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 files
fps = 30

# Assume first frame to get video dimensions
first_frame_path = os.path.join(frames_dir, '000001.jpg')
first_frame = cv2.imread(first_frame_path)
frame_height, frame_width = first_frame.shape[:2]
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process each frame
sorted_frame_files = sorted(os.listdir(frames_dir))  # Sort to ensure correct frame order
for frame_file in sorted_frame_files:
    if frame_file.endswith('.jpg'):  # Check for jpeg images
        frame_path = os.path.join(frames_dir, frame_file)
        frame = cv2.imread(frame_path)
        if frame is None:
            continue
        video_writer.write(frame)

# Release the VideoWriter
video_writer.release()
print(f"Video saved to {output_video_path}")

Video saved to output_videos/sensor_2_right.mp4


In [ ]:
ls

BoT-SORT/                 streetaware_sample_chase_1_sensor_1_left/
bot_sort.ipynb            streetaware_sample_chase_1_sensor_1_right/
bytetrack/                streetaware_sample_chase_1_sensor_2_left/
bytetrack_output_frames/  streetaware_sample_chase_1_sensor_2_right/
data_preprocessing.ipynb  streetaware_sample_chase_2_sensor_1_left/
input/                    streetaware_sample_chase_2_sensor_1_right/
MOT17/                    streetaware_sample_chase_2_sensor_2_left/
output_frames/            streetaware_sample_chase_2_sensor_2_right/
outputs/                  yolov8l.pt
output_videos/            yolov8n.pt
StreetAware/              yolov8x.pt


In [13]:
def box_overlap_count(boxes):
    def overlap(box1, box2):
        # Check if box1 overlaps with box2
        x1_min, y1_min, x1_max, y1_max = box1
        x2_min, y2_min, x2_max, y2_max = box2
        return not (x1_max < x2_min or x2_max < x1_min or y1_max < y2_min or y2_max < y1_min)

    count = 0
    for i in range(len(boxes)):
        for j in range(i + 1, len(boxes)):
            if overlap(boxes[i], boxes[j]):
                count += 1
    return count


In [42]:
import json

video_name = "streetaware_sample_chase_2_sensor_2_right"
txt_path = 'outputs/botsort/chase_2/sensor_2/right_people_track.txt'
output_json = 'outputs/botsort/chase_2/sensor_2/right_people_track.json'
frame_data = {}

# Reading data from file
with open(txt_path, 'r') as file:
    data = file.read()

for line in data.strip().split("\n"):
    parts = line.split(',')
    frame_id = int(parts[0])
    people_id = int(parts[1])
    x1 = float(parts[2])
    y1 = float(parts[3])
    width = float(parts[4])
    height = float(parts[5])
    conf = float(parts[6])
    x2 = x1 + width
    y2 = y1 + height
    bbox_area = width * height
    if frame_id not in frame_data:
        frame_data[frame_id] = []
    frame_data[frame_id].append([x1, y1, x2, y2, people_id, bbox_area, conf])

In [43]:

json_list = []

# Process each frame to calculate overlaps
for frame_id, boxes in frame_data.items():
    bboxes = [[box[0], box[1], box[2], box[3]] for box in boxes]
    overlap_counts = box_overlap_count(bboxes)

    for box in boxes:
        json_obj = {
            "video": video_name,
            "frame": frame_id,
            "people_id": box[4],
            "bbox": [round(box[0], 2), round(box[1], 2), round(box[2], 2), round(box[3], 2)],
            "bbox_area": round(box[5],2),
            "n_overlapping_boxes": overlap_counts,
            "score":  box[6]
        }
        json_list.append(json_obj)

# Convert list to JSON and write to file
json_data = json.dumps(json_list, indent=4)
with open(output_json , 'w') as json_file:
    json_file.write(json_data)